# Notebook 2: Geometric Brownian Motion for Stock Prices

**Learning Objectives:**
- Understand why stock returns are modeled as log-normal
- Implement Geometric Brownian Motion (GBM)
- Generate thousands of possible market futures
- Analyze the distribution of outcomes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

## The GBM Formula

$$S_{t+1} = S_t \cdot \exp\left((\mu - \frac{\sigma^2}{2})\Delta t + \sigma\sqrt{\Delta t} \cdot Z\right)$$

Where:
- $\mu$ = expected annual return (drift), e.g., 0.07 for 7%
- $\sigma$ = annual volatility, e.g., 0.16 for 16%
- $Z$ = standard normal random variable

In [ ]:
def simulate_gbm_paths(S0, mu, sigma, T, n_paths, seed=None):
    """Generate multiple GBM price paths."""
    if seed is not None:
        np.random.seed(seed)
    dt = 1.0
    Z = np.random.standard_normal((n_paths, T))
    log_returns = (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
    cumulative = np.cumsum(log_returns, axis=1)
    paths = np.zeros((n_paths, T + 1))
    paths[:, 0] = S0
    paths[:, 1:] = S0 * np.exp(cumulative)
    return paths

# Generate 1000 30-year paths
paths = simulate_gbm_paths(S0=1_000_000, mu=0.07, sigma=0.16, T=30, n_paths=1000, seed=42)
print(f"Generated {paths.shape[0]} paths of {paths.shape[1]-1} years each")

In [ ]:
# Visualize paths
plt.figure(figsize=(12, 6))
for i in range(100):
    plt.plot(paths[i]/1e6, alpha=0.1, color='blue')
plt.plot(np.median(paths, axis=0)/1e6, 'r-', lw=2, label='Median')
plt.plot(np.percentile(paths, 10, axis=0)/1e6, 'g--', lw=2, label='10th/90th pctl')
plt.plot(np.percentile(paths, 90, axis=0)/1e6, 'g--', lw=2)
plt.xlabel('Year'); plt.ylabel('Portfolio Value ($M)')
plt.title('1000 Simulated 30-Year Portfolio Paths'); plt.legend(); plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Terminal value statistics
terminal = paths[:, -1]
print("Terminal Portfolio Value Statistics (Year 30)")
print(f"Mean:   ${np.mean(terminal):>12,.0f}")
print(f"Median: ${np.median(terminal):>12,.0f}")
print(f"5th:    ${np.percentile(terminal, 5):>12,.0f}")
print(f"95th:   ${np.percentile(terminal, 95):>12,.0f}")

In [ ]:
# Probability questions
print(f"P(double money): {np.mean(terminal >= 2e6)*100:.1f}%")
print(f"P(5x money):     {np.mean(terminal >= 5e6)*100:.1f}%")
print(f"P(lose money):   {np.mean(terminal < 1e6)*100:.1f}%")

In [ ]:
# Impact of volatility
print("Impact of Volatility on 30-Year Outcomes")
print(f"{'Vol':>6} {'Median':>12} {'5th Pctl':>12} {'95th Pctl':>12}")
for sigma in [0.10, 0.16, 0.22, 0.30]:
    p = simulate_gbm_paths(1e6, 0.07, sigma, 30, 5000, seed=42)
    t = p[:, -1]
    print(f"{sigma:>6.0%} ${np.median(t)/1e6:>11.2f}M ${np.percentile(t,5)/1e6:>11.2f}M ${np.percentile(t,95)/1e6:>11.2f}M")

## Summary

- GBM models stock prices with drift (expected return) and diffusion (volatility)
- Higher volatility = wider range of outcomes
- We can answer probabilistic questions by counting simulation outcomes

**Next: Notebook 3 - Survival Analysis** (how long will the retiree live?)